In [25]:
import numpy as np
import pandas as pd
import os,sys

In [26]:
path_data = os.path.join(os.getcwd(),os.pardir,os.pardir,'data')
data_list = os.listdir(path_data)

In [27]:
protected_area_data_path = os.path.join(path_data,'protected_area')
# data_list = os.listdir(protected_area_data_path)
protected_area_data_file = os.path.join(protected_area_data_path,'NOAA_Marine_Protected_Areas_Inventory_2023.xlsx')
protected_area_data = pd.read_excel(protected_area_data_file)
protected_area_data = protected_area_data[['Percent Area Marine','Year Established','Longitude','Latitude']]
print(len(protected_area_data))

979


In [28]:
start_year = 2008
year_filtered_protected_area_data = protected_area_data[protected_area_data['Year Established'] < start_year].copy()


In [29]:
import pandas as pd

lat_min, lat_max = -80, 80
lon_min, lon_max = -180, 180
# Define the bin size for latitude and longitude
lat_bin_size = 5
lon_bin_size = 5

def calculate_bin(value, bin_size):
    bin_center = ((value + bin_size/2) // bin_size) * bin_size
    return bin_center

lat_bins = np.arange(lat_min, lat_max + lat_bin_size, lat_bin_size)
lon_bins = np.arange(lon_min, lon_max + lon_bin_size, lon_bin_size)

# Bin the data for the entire dataset
year_filtered_protected_area_data['lat_bin'] = pd.cut(year_filtered_protected_area_data['Latitude'], bins=lat_bins)
year_filtered_protected_area_data['lon_bin'] = pd.cut(year_filtered_protected_area_data['Longitude'], bins=lon_bins)

# Create a column to count occurrences where Percent_Bleaching > 0
# year_filtered_protected_area_data['Percent Area Marine'] = year_filtered_protected_area_data['Percent Area Marine'] > 0
# Group by latitude and longitude bins, then sum the 'Percent Area Marine' values
mpa_counts = year_filtered_protected_area_data.groupby(['lat_bin', 'lon_bin'])['Percent Area Marine'].sum().reset_index(name='count')

# Save the aggregated data
mpa_counts.to_csv(os.path.join(protected_area_data_path, 'mpa_counts.csv'), index=False)


In [30]:
save_fig_folder = os.path.join(os.getcwd(),os.pardir,'figures')
if not os.path.exists(save_fig_folder):
    os.makedirs(save_fig_folder)

In [31]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Sort the data by 'count' in descending order and select the top 10
top_mpa_counts = mpa_counts.sort_values(by='count', ascending=False).head(10)
fontsize = 24
fontsize_title = 25
# Plotting
plt.figure(figsize=(40, 10))
plt.barh(top_mpa_counts['lat_bin'].astype(str) + ', ' + top_mpa_counts['lon_bin'].astype(str), 
         top_mpa_counts['count'])
plt.xlabel('Count', fontsize=fontsize)
plt.ylabel('Latitude, Longitude Bins', fontsize=fontsize)
plt.title('Top 10 Locations with Highest MPA Counts during 2008-2018', fontsize=fontsize_title)
plt.gca().invert_yaxis()  # Invert y-axis to have the highest count at the top
plt.tick_params(axis='both', which='major', labelsize=20)
filename = 'top_10_locations_with_highest_mpa_counts.png'
plt.savefig(os.path.join(save_fig_folder,filename))
plt.show()

In [32]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

# Continue from your code
fontsize = 24
fontsize_title = 25

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 10))

# Create a Basemap instance
m = Basemap(projection='merc', llcrnrlat=lat_min, urcrnrlat=lat_max,
            llcrnrlon=lon_min, urcrnrlon=lon_max, resolution='i', ax=ax)

# Draw coastlines and countries
m.drawcoastlines()
m.drawcountries()

# Get central points of the top 10 bins
# Assuming 'top_bleaching_counts' is your DataFrame with the top 10 bins
for idx, row in top_mpa_counts.iterrows():
    lat_center = row['lat_bin'].mid
    lon_center = row['lon_bin'].mid

    # Convert lat and lon to map projection coordinates
    x, y = m(lon_center, lat_center)

    # Plot the point on the map
    m.plot(x, y, color = 'blue', marker='o', markersize=10) 

    # Optionally, you can add a label with the count
    plt.text(x, y, str(row['count']), fontsize=15, ha='right', va='bottom', color='red')

plt.title("Top 10 Locations with Highest MPA Counts furing 2008-2018", fontsize=fontsize_title)

filename = os.path.join(save_fig_folder,'top10_mpa_counts.png')
plt.savefig(filename)
plt.show()
